### Convert the .dcm image to .nii.gz image in BraTS2021 segmentation.

- .dcm image: BraTS2021 Task 2 (MGMT classification).
- .nii.gz image: BraTS2021 Task 1 (Segmentation).

**Preprocessing:**
- conversion of the DICOM files to the NIFTI file format; 
- re-orientation to a common orientation system (i.e., RAI);
- co-registration to the same anatomical template (SRI24);
- resampling to a uniform isotropic resolution (1mm3);
- finally skull-stripping.

**Orientation**
- dcm to nii directly: 'LAS';
- template: 'LAS';
- registered image: 'LAS';
- BraTS2021 segmentation image: 'LPS';

The preprocessing pipeline is publicly available through the Cancer Imaging Phenomics Toolkit (CaPTk) and Federated Tumor Segmentation (FeTS) tool.

**References:**
- See more details in paper "The RSNA-ASNR-MICCAI BraTS 2021 Benchmark on Brain Tumor Segmentation and Radiogenomic Classification".
- https://www.kaggle.com/ranafago/preprocessing-dcm-to-nifti


In [ ]:
import os
import gc
import ants
import nibabel as nib
import dicom2nifti
import SimpleITK as sitk
import  matplotlib.pyplot as plt
import nibabel.processing as nibabel_processing

import sys
sys.path.append("E://2020_MRI_Work/HarmonizationProject")
from utils.myUtils import get_filenames, mkdir, traversalDir_FirstDir


from myFeatureExtractionSettings import get_sri24_template_path_dict, get_image_path_dict, get_modality_dict

In [ ]:
'''
Register images and save the results.
'''
def register_images(fixed_image_path, moving_image_path, save_registered_dir):
    print("\n register images: \n fix_image_path={}, \n moving_image_path={}.".format(fixed_image_path, moving_image_path))
    
    fixed_img = ants.image_read(fixed_image_path)
    moving_img = ants.image_read(moving_image_path)
    registered_results= ants.registration(fixed=fixed_img, moving=moving_img, type_of_transform = 'Rigid' )
    registered_moving_image =registered_results['warpedmovout']
    transform_from_move_to_fix=registered_results['fwdtransforms']
    transform_from_fix_to_move=registered_results['invtransforms']
    
    
    #resave the fixed image and the registered moving image.
    resave_registered_fixed_image_path=save_registered_dir +'/' + os.path.basename(fixed_image_path)
    resave_registered_moving_image_path=save_registered_dir +'/' + os.path.basename(moving_image_path)
    #print('\n fixed_img=',fixed_img)
    #print('\n registered_moving_image=',registered_moving_image)
    ##ants.image_write(fixed_img, resave_registered_fixed_image_path)
    ants.image_write(registered_moving_image, resave_registered_moving_image_path)
    
    return  resave_registered_fixed_image_path, resave_registered_moving_image_path


'''
Visualize the nii results.
'''
def show_nii_images(nii_images_dict, save_visualize_basic_path):
    
    load_nii_images={}
    for image_name, image_path in nii_images_dict.items():
        load_nii_images[image_name]=nib.load(image_path).get_fdata()
    
    #show the images
    number_of_slices=list(load_nii_images.values())[0].shape[2]
    for index in range(0, number_of_slices):
    
        num_figures=len(load_nii_images)
        plt.figure(figsize=(5*num_figures,5))
        j=0
        for image_name, nii_image in load_nii_images.items():
            j=j+1
            sub_fig=plt.subplot(1,num_figures,j)
            sub_fig.set_title(image_name, fontsize=20)
            plt.axis('off')
            plt.imshow(nii_image[:,:,index].T)


        plt.subplots_adjust(left=0.03, bottom=0.03, right=0.97, top=0.9,  wspace=0.03, hspace=0)
        
        basename=os.path.basename(image_path)
        save_path= save_visualize_basic_path+'/'+ basename[:-5]+'_'+str(index)+'.png'
        plt.savefig(save_path)
        
        plt.show()
        
        # Clear the current axes.
        plt.cla() 
        # Clear the current figure.
        plt.clf() 
        # Closes all the figure windows.
        plt.close('all')
        gc.collect()

        
def get_nii_image_orientation(nii_image_path):

    nii_img = nib.load(nii_image_path)
    otientation = nib.aff2axcodes(nii_img.affine)
    print("\n nii_image_path={}, otientation={}".format(nii_image_path, otientation))
    
    return otientation

In [ ]:
def preprocess_dcm_to_nii(dcm_image_basepath, save_nii_image_basepath, patient_id, orig_nii_image_basepath=None, show_images=False):
    #print("- patient_id: {}".format(patient_id))
    
    dcm_image_path=os.path.join(dcm_image_basepath, patient_id)
    save_nii_image_path=os.path.join(save_nii_image_basepath, "1_dcm_to_nii", "BraTS2021_"+patient_id)
    save_resampled_nii_image_path=os.path.join(save_nii_image_basepath, "2_resampled_nii", "BraTS2021_"+patient_id)
    save_registered_nii_image_path=os.path.join(save_nii_image_basepath, "3_registered_nii", "BraTS2021_"+patient_id)
    mkdir(save_nii_image_path)
    mkdir(save_resampled_nii_image_path)
    mkdir(save_registered_nii_image_path)
    
    # Modality names.
    modality_dict=get_modality_dict()

    for dcm_modality, nii_modality in modality_dict.items():
        print("\n - Modality: {}".format(dcm_modality))
        
        dcm_image_folder=os.path.join(dcm_image_path, dcm_modality)
        base_name="BraTS2021_"+patient_id+"_"+nii_modality+".nii.gz"
        save_nii_modality_image_path=os.path.join(save_nii_image_path, base_name)
        
        # 1.conversion of the DICOM files to the NIFTI file format;
        dicom2nifti.settings.enable_validate_slice_increment()
        dicom2nifti.settings.enable_validate_orthogonal()
        try:
            dicom2nifti.dicom_series_to_nifti(dcm_image_folder, save_nii_modality_image_path, reorient_nifti=True) # LAS oriented
        
        except Exception as error:
            print("Caution!!! patient_id={}, modality={}, error: {}.".format(patient_id, nii_modality, error))
            file = r'./dcm_to_nii_error_list.txt'
            with open(file, 'a+') as f:
                 f.write("patient_id={}, modality={}, error: {}. \n".format(patient_id, nii_modality, error)) 
            
            dicom2nifti.settings.disable_validate_slice_increment()
            dicom2nifti.settings.disable_validate_orthogonal()
            dicom2nifti.dicom_series_to_nifti(dcm_image_folder, save_nii_modality_image_path, reorient_nifti=True) # LAS oriented
          
        
        nii_img=nib.load(save_nii_modality_image_path)
        print("Shape of the original nii image:{}".format(nii_img.shape))
        
        # 2. resampling the image to 240x240x155, 1mm x 1mm x 1mm, and reorientation.
        nii_img_resampled_reoriented=nibabel_processing.conform(nii_img, out_shape=(240, 240, 155), voxel_size=(1.0, 1.0, 1.0), orientation='LAS')
        save_resampled_nii_modality_image_path=os.path.join(save_resampled_nii_image_path, base_name)
        nib.save(nii_img_resampled_reoriented, save_resampled_nii_modality_image_path)
        print("Shape of the resampled and reoriented image:{}".format(nii_img_resampled_reoriented.shape))
        
        
        # 3.1. co-registration to the same anatomical template (SRI24);
        sri24_template_path_dict=get_sri24_template_path_dict()
        sri24_t1_template=sri24_template_path_dict["sri24_t1_template"]
        sri24_t2_template=sri24_template_path_dict["sri24_t2_template"]
        
        #sri24_template=sri24_t1_template if (nii_modality=="t1" or nii_modality=="t1ce") else sri24_t2_template
        sri24_template=sri24_t2_template
        _, save_registered_nii_modality_image_path=register_images(sri24_template, save_resampled_nii_modality_image_path, save_registered_nii_image_path)
    
      
       # 3.2. reorientated the registered images to the orientation used in BraTS2021, from "LAS" to "LPS".  
        registered_nii_modality_image=nib.load(save_registered_nii_modality_image_path)
        registered_nii_modality_image_orientated=nibabel_processing.conform(registered_nii_modality_image, out_shape=(240, 240, 155), voxel_size=(1.0, 1.0, 1.0), orientation='LPS')
        nib.save(registered_nii_modality_image_orientated, save_registered_nii_modality_image_path)
        
       # 4. compare the preprocessed and registered .nii images with the .nii images provided by the BraTS2021 segmentation task.
        if show_images:
            save_visualize_basic_path=os.path.join(save_nii_image_basepath, "4_visualization", dcm_modality)
            mkdir(save_visualize_basic_path)
            nii_images_dict={"resampled images":save_resampled_nii_modality_image_path,
                             "registered images": save_registered_nii_modality_image_path}
            
            if orig_nii_image_basepath is not None:
                orig_nii_modality_image_path=os.path.join(orig_nii_image_basepath, "BraTS2021_"+patient_id, base_name)
                nii_images_dict["original nii images"]=orig_nii_modality_image_path
                
            show_nii_images(nii_images_dict, save_visualize_basic_path)


### Main function for converting .dcm to .nii images.

In [ ]:
# ### Examples

# patient_id="00000"
# dcm_image_basepath="E://example_BraTS2021/orig_dcm"
# save_nii_image_basepath="E://example_BraTS2021/dcm_to_nii"
# orig_nii_image_basepath="E://example_BraTS2021/orig_nii"

# preprocess_dcm_to_nii(dcm_image_basepath, save_nii_image_basepath, patient_id, orig_nii_image_basepath)

In [ ]:
data_type="train" #"train", "validation"
image_path_dict=get_image_path_dict()
dcm_image_path=image_path_dict[data_type+"_dcm_path"]
original_nii_image_path=image_path_dict[data_type+"_nii_path"]
save_nii_image_path=image_path_dict[data_type+"_dcm_to_nii_path"]

patient_id_list=traversalDir_FirstDir(dcm_image_path)
num_patients=len(patient_id_list)
print("There are totally {} patients in {} data.".format(num_patients, data_type))


for i in range(0, len(patient_id_list)):
    patient_id=patient_id_list[i]
    print("\n\n =================== {}/{}: {}====================".format(i+1, num_patients, patient_id))
    preprocess_dcm_to_nii(dcm_image_path, save_nii_image_path, patient_id, original_nii_image_path)